#Configuration

## Set working directory and mount Google drive

In [1]:
#  For only colab
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

    # To access contents in Google drive
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#  source root directory
src_dir = '/content/drive/MyDrive/projects/genai_workshop/notebook/03.cloud function/functions'

# cloud function src directory.
import os
os.chdir(src_dir)
!pwd

/content/drive/MyDrive/projects/genai_workshop/notebook/03.cloud function/functions


# Cloud function

In [4]:
from google.colab import files
files.view(f'{src_dir}/main.py')
files.view(f'{src_dir}/requirements.txt')
files.view(f'{src_dir}/rag/constant.py')
files.view(f'{src_dir}/rag/controller.py')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#%%writefile table_search.py

Writing table_search.py


## Copy common codes from directory.

In [5]:
!gcloud auth application-default login
!gcloud config set project ai-hangsik
!gcloud auth application-default set-quota-project ai-hangsik


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=arD0kqSXdTQVNE1k7nHdGg81M4fWqY&prompt=consent&token_

In [ ]:
# !gcloud auth login


In [6]:
!pwd

/content/drive/MyDrive/projects/genai_workshop/notebook/03.cloud function/functions


## Deploy Cloud Run - Text Processing

In [15]:
!gcloud functions deploy it_search --allow-unauthenticated \
	--region=asia-northeast3 \
	--gen2 \
	--memory=8Gi \
	--cpu=4 \
	--concurrency=4 \
	--min-instances=2 \
	--max-instances=5 \
	--timeout=30s \
	--runtime=python312 \
  --trigger-http \
	--source=. \
	--entry-point=search


  [INFO] A new revision will be deployed serving with 100% traffic.
You can view your function in the Cloud Console here: https://console.cloud.google.com/functions/details/asia-northeast3/it_search?project=ai-hangsik

buildConfig:
  automaticUpdatePolicy: {}
  build: projects/721521243942/locations/asia-northeast3/builds/5947b816-fa6f-492d-af18-e7976a802ecb
  dockerRegistry: ARTIFACT_REGISTRY
  dockerRepository: projects/ai-hangsik/locations/asia-northeast3/repositories/gcf-artifacts
  entryPoint: search
  runtime: python312
  source:
    storageSource:
      bucket: gcf-v2-sources-721521243942-asia-northeast3
      generation: '1724760315825580'
      object: it_search/function-source.zip
  sourceProvenance:
    resolvedStorageSource:
      bucket: gcf-v2-sources-721521243942-asia-northeast3
      generation: '1724760315825580'
      object: it_search/function-source.zip
createTime: '2024-08-19T18:57:47.726297631Z'
environment: GEN_2
labels:
  deployment-tool: cli-gcloud
name: projec

## REST API Call to Cloud function.

In [14]:
!curl -X POST https://it-search-721521243942.asia-northeast3.run.app \
-H 'Content-Type: application/json' \
-d '{"question":"개인정보 보호법에 대해서 설명해주세요."}'

500 Internal Server Error: The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.